- compare the FR numbers from here to those of Hopkins
- same for departement data
- pareil pour les "en réa"

Les données proviennent de https://github.com/opencovid19-fr/data.

Ce notebook a été largement inspiré par https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-death-trajectories.ipynb.

In [1]:
import pandas as pd
import numpy as np
import yaml
import altair as alt
#import seaborn as sns
#import matplotlib.pyplot as plt
from IPython.display import HTML

In [2]:
def make_since_chart(dff2, highlight_countries=[], baseline_countries=[], Y_to_plot=[], scale_type='log', title = ''):
    selection = alt.selection_multi(fields=['maille_nom'], bind='legend', 
                                    init=[{'maille_nom': x} for x in highlight_countries + baseline_countries])

    base = alt.Chart(dff2, width=550).encode(
        x=f'{COL_X}:Q',
        y=alt.Y('{}:Q'.format(Y_to_plot), scale=alt.Scale(type=scale_type), axis=alt.Axis(title='Nombre cumulé')),
        color=alt.Color('maille_nom:N', scale=alt.Scale(domain=color_domain, range=color_range)),
        tooltip=list(dff2), # to have info w/ hover
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
    max_day = dff2[COL_X].max()
    ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, Y_to_plot])
    base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='{}:Q'.format(Y_to_plot))
    return (
        alt.layer(
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]),
        base_ref.transform_filter(
            alt.datum[COL_X] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=10, text='+33% par jour'),
        base.mark_line(point=True).add_selection(selection),
        base.transform_filter(
            alt.datum['date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='maille_nom:N')
        )
    ).properties(
#        title=f"Compare {', '.join(highlight_countries)} death trajectory with {', '.join(baseline_countries)}"
        title=f"{title}"
    )

In [3]:
#file = ('./data/agences-regionales-sante/'
#       'auvergne-rhone-alpes/2020-02-27.yaml')
#fruits_list = yaml.load(open(file), Loader=yaml.FullLoader)

In [4]:
csv_filename = './data/dist/chiffres-cles.csv'
df = pd.read_csv(csv_filename)

In [5]:
df.date = pd.to_datetime(df.date, format='%Y-%m-%d')
dff = df.loc[(df.granularite=='region') & (df['source_nom'].str.contains("ARS")),
            ['date', 'maille_nom', 'deces', 'cas_confirmes', 'reanimation']]

In [6]:
#dff = dff.dropna() 
myregions = np.unique(dff.maille_nom)
dt_cols = np.unique(dff.date)

In [7]:
LAST_DATE = dff.date.max()
LAST_DATE

Timestamp('2020-03-23 00:00:00')

## Opendatasoft

https://public.opendatasoft.com/explore/dataset/evolution-de-la-pandemie-de-covid-19-en-france/export/

In [8]:
# cas confirmés uniquement
# ods_dataset_id = 'evolution-de-la-pandemie-de-covid-19-en-france'
filename = 'evolution-de-la-pandemie-de-covid-19-en-france.csv'
df_ods_confirmes = pd.read_csv(filename, sep=';')

In [9]:
df_ods_confirmes['date'] = pd.to_datetime(df_ods_confirmes['Date'])
df_ods_confirmes['cas_confirmes'] = df_ods_confirmes['Nombre de cas confirmés']
df_ods_confirmes['maille_nom'] = df_ods_confirmes['Nom officiel']
df_ods_confirmes = df_ods_confirmes[['date','maille_nom', 'cas_confirmes']]
df_ods_confirmes = df_ods_confirmes.sort_values(by='date')

## Analyse

In [27]:
#hide
MIN_CONFIRMES = 2
LAST_DATE = dt_cols[-1]

#myregion = dff[dff['date'].eq(LAST_DATE) & dff['deces'].ge(MIN_CASES)
#       ].sort_values(by='deces', ascending=False)
myregion = dff[dff['cas_confirmes'].ge(MIN_CONFIRMES)
       ].sort_values(by='cas_confirmes', ascending=False)
myregion = myregion['maille_nom'].values
myregion = np.unique(myregion)

In [28]:
#hide
def get_country_colors(x):
    mapping = {
        'Auvergne-Rhône-Alpes': '#A1BA59',
        'Bourgogne-Franche-Comté': '#A1BA59',
        'Bretagne': '#9467bd',
        'Centre-Val de Loire': '#9D755D',
        'Corse': '#2495D3', 
        'Grand-Est': 'black', 
        'Guadeloupe': '#C1B7AD',
        'Guyane': '#C1B7AD', 
        'Hauts-de-France': '#2495D3', 
        'Ile-de-France': 'black', 
        'Martinique': '#C1B7AD',
        'Mayotte': '#C1B7AD',
        'Normandie': '#C1B7AD',
        'Nouvelle-Aquitaine': 'black', 
        'Occitanie': '#E45756',
        'Pays de la Loire': '#F58518',
        'Provence-Alpes-Côte d’Azur': 'black'}
    return mapping.get(x, '#C1B7AD')

# Décès

In [29]:
#hide
SINCE_CASES_NUM = 8
COL_X = f'Days since {SINCE_CASES_NUM}th death'
dff2 = dff[dff['maille_nom'].isin(myregions)].copy()
days_since = (dff2.assign(F=dff2['deces'].ge(SINCE_CASES_NUM))
              .set_index('date')
              .groupby('maille_nom')['F'].transform('idxmax'))
dff2[COL_X] = (dff2['date'] - days_since.values).dt.days.values
dff2 = dff2[dff2[COL_X].ge(0)]

# Need to remove 0 to get the log scale to not generate -inf
dff2.deces[dff2.deces==0]=np.nan

In [33]:
dff2[dff2.maille_nom=='La réunion']

,date,maille_nom,deces,cas_confirmes,reanimation,Days since 8th death


In [31]:
#hide_input
baseline_countries = ['Occitanie','Hauts-de-France','Grand-Est']
max_date = dff2['date'].max()
color_domain = list(dff2['maille_nom'].unique())
color_range = list(map(get_country_colors, color_domain))

chart = make_since_chart(dff2, highlight_countries=[], 
                         baseline_countries=baseline_countries, 
                         Y_to_plot='deces', scale_type='log',
                         title='Trajectoires du nombre de décès')
chart

alt.LayerChart(...)

# Confirmed cases

In [14]:
#hide
SINCE_CASES_NUM = 20
COL_X = f'Days since {SINCE_CASES_NUM}th case'
dff3 = dff[dff['maille_nom'].isin(myregions)].copy()
days_since = (dff3.assign(F=dff3['cas_confirmes'].ge(SINCE_CASES_NUM))
              .set_index('date')
              .groupby('maille_nom')['F'].transform('idxmax'))
dff3[COL_X] = (dff3['date'] - days_since.values).dt.days.values
dff3 = dff3[dff3[COL_X].ge(0)]

dff3.cas_confirmes[dff3.cas_confirmes==0]=np.nan

In [15]:
dff3[dff3.maille_nom=='Hauts-de-France']

,date,maille_nom,deces,cas_confirmes,reanimation,Days since 20th case
963,2020-03-09,Hauts-de-France,15.0,181.0,NaN,0
1091,2020-03-10,Hauts-de-France,16.0,222.0,NaN,1
1371,2020-03-12,Hauts-de-France,20.0,337.0,NaN,3
1491,2020-03-13,Hauts-de-France,26.0,368.0,NaN,4
1618,2020-03-14,Hauts-de-France,29.0,463.0,NaN,5
1845,2020-03-16,Hauts-de-France,39.0,587.0,NaN,7
1960,2020-03-17,Hauts-de-France,46.0,613.0,NaN,8
2180,2020-03-18,Hauts-de-France,55.0,668.0,NaN,9
2412,2020-03-19,Hauts-de-France,69.0,719.0,NaN,10
2629,2020-03-20,Hauts-de-France,81.0,740.0,NaN,11


In [16]:
#hide_input
baseline_countries = ['Occitanie','Hauts-de-France','Grand-Est']
max_date = dff3['date'].max()
color_domain = list(dff3['maille_nom'].unique())
color_range = list(map(get_country_colors, color_domain))

chart = make_since_chart(dff3, highlight_countries=[], 
                         baseline_countries=baseline_countries, 
                         Y_to_plot='cas_confirmes', 
                         title='Trajectoires du nombre de cas')
chart

alt.LayerChart(...)